In [9]:
from diagrams import Diagram, Cluster
from diagrams.aws.compute import Lambda
from diagrams.aws.storage import S3
from diagrams.aws.database import DynamodbAttribute
from diagrams.aws.database import DynamodbAttributes
from diagrams.aws.database import Dynamodb
from diagrams.aws.database import DynamodbGlobalSecondaryIndex
from diagrams.aws.database import DynamodbItem
from diagrams.aws.database import DynamodbItems
from diagrams.aws.database import DynamodbTable
from diagrams.generic.blank import Blank
from diagrams.custom import Custom
from diagrams.aws.management import SystemsManagerParameterStore
from diagrams.aws.management import SSM


In [ ]:
with Diagram(filename="primer_prototipo", show=False):
    with Cluster("DynamoDB"):
        authors_table = DynamodbTable("authors")
    (
        Lambda("GetBooks")
        >> S3("authors-and-books")
        >> Lambda("InsertBooks")
        >> authors_table
    )


In [10]:
with Diagram(filename="segundo_prototipo", show=False):
    with Cluster("Get Books"):
        itbooks = Lambda("IT Books")
        nytimes = Lambda("NY Times")
    with Cluster("DynamoDB"):
        authors_table = DynamodbTable("authors")

    s3_bucket = S3("authors-and-books")
    api_token = SystemsManagerParameterStore("API Token")
    api_token - nytimes

    itbooks >> s3_bucket
    nytimes >> s3_bucket
    s3_bucket >> Lambda("InsertBooks") >> authors_table


In [16]:
with Diagram(filename="tercer_prototipo", show=False):
    with Cluster("Prefect"):
        with Cluster("Get Books"):
            itbooks = Lambda("IT Books")
            nytimes = Lambda("NY Times")

        with Cluster("DynamoDB"):
            authors_table = DynamodbTable("authors")

        s3_bucket = S3("authors-and-books")

        itbooks >> s3_bucket
        nytimes >> s3_bucket
        s3_bucket >> Lambda("InsertBooks") >> authors_table
        # >> Lambda("InsertBooks") >> Dynamodb() >> DynamodbTable("authors")

    api_token = SystemsManagerParameterStore("API Token")
    api_token - nytimes


In [17]:
with Diagram(filename="cuarto_prototipo", show=False):
    with Cluster("Prefect"):
        with Cluster("API Books"):
            api = [Lambda("IT Books"), Lambda("NY Times")]

        with Cluster("Biblioteca Books"):
            get_books = Lambda("Get Books")
            clean_books = Lambda("Clean Books")
            biblioteca = get_books >> clean_books

        with Cluster("DynamoDB"):
            authors_table = DynamodbTable("authors")

        s3_bucket = S3("authors-and-books")

        api >> s3_bucket
        biblioteca >> s3_bucket
        s3_bucket >> Lambda("InsertBooks") >> authors_table
    # >> Lambda("InsertBooks") >> Dynamodb() >> DynamodbTable("authors")
    api_token = SystemsManagerParameterStore("API Token")
    api_token - api[1]


In [13]:
with Diagram(filename="quinto_prototipo", show=False):
    with Cluster("Prefect"):
        with Cluster("API Books"):
            api = [Lambda("IT Books"), Lambda("NY Times")]

        with Cluster("Biblioteca Books"):
            get_books = Lambda("Get Books")
            clean_books = Lambda("Clean Books")
            biblioteca = get_books >> clean_books

        with Cluster("DynamoDB"):
            authors_table = DynamodbTable("authors")

        s3_bucket = S3("authors-and-books")
        api_token - nytimes

        api >> s3_bucket
        biblioteca >> s3_bucket
        s3_bucket >> Lambda("InsertBooks") >> authors_table

    with Cluster("DynamoDB Stream"):
        with Cluster("DynamoDB"):
            metadata_table = DynamodbTable("matadata")

        authors_table >> Lambda("UpdateMetadata") >> metadata_table

    api_token = SystemsManagerParameterStore("API Token")
    api_token - api[1]
